In [1]:
%install '.package(path: "$cwd/YOLOv4")' YOLOv4
%install '.package(url: "https://github.com/tensorflow/swift-models", .branch("tensorflow-0.9"))' ModelSupport
import TensorFlow
import YOLOv4
import ModelSupport
import Foundation

Installing packages:
	.package(path: "/notebooks/YOLOv4")
		YOLOv4
	.package(url: "https://github.com/tensorflow/swift-models", .branch("tensorflow-0.9"))
		ModelSupport
With SwiftPM flags: []
Working in: /tmp/tmplgknqkw3/swift-install
Fetching https://github.com/tensorflow/swift-models
Fetching https://github.com/apple/swift-protobuf.git
Fetching https://github.com/apple/swift-argument-parser
Cloning https://github.com/tensorflow/swift-models
Resolving https://github.com/tensorflow/swift-models at tensorflow-0.9
Cloning https://github.com/apple/swift-argument-parser
Resolving https://github.com/apple/swift-argument-parser at 0.0.5
Cloning https://github.com/apple/swift-protobuf.git
Resolving https://github.com/apple/swift-protobuf.git at 1.8.0
[1/25] Compiling STBImage stb_image_write.c
[2/97] Compiling STBImage stb_image.c
[3/97] Compiling SwiftProtobuf BinaryDecodingOptions.swift
/notebooks/YOLOv4/Sources/HackyOps.swift:11:27: warning: redundant conformance constraint 'Features': 'D

    public var activ: Activation = mish
    ^
    @noDerivative 
/notebooks/YOLOv4/Sources/HackyOps.swift:11:27: warning: redundant conformance constraint 'Features': 'Differentiable'
public func |> <Features: Differentiable, Func: Layer>(left: Func.Input, right:Func) -> Func.Output where Features == Func.Input {
                          ^
/notebooks/YOLOv4/Sources/HackyOps.swift:11:49: note: conformance constraint 'Features': 'Differentiable' implied here
public func |> <Features: Differentiable, Func: Layer>(left: Func.Input, right:Func) -> Func.Output where Features == Func.Input {
                                                ^
[28/97] Compiling SwiftProtobuf AnyUnpackError.swift
/notebooks/YOLOv4/Sources/Basic.swift:7:5: warning: stored property 'useZero' has no derivative because 'Bool' does not conform to 'Differentiable'; add an explicit '@noDerivative' attribute
    public var useZero: Bool = false
    ^
    @noDerivative 
/notebooks/YOLOv4/Sources/Basic.swift:8:5: warning:

  var readBuffer = UnsafeMutablePointer<UInt8>.allocate(capacity: 1)
  ~~~ ^
  let
[66/97] Compiling SwiftProtobuf ExtensionFieldValueSet.swift
/tmp/tmplgknqkw3/swift-install/package/.build/checkouts/swift-protobuf/Sources/SwiftProtobuf/BinaryDelimited.swift:198:7: warning: variable 'readBuffer' was never mutated; consider changing to 'let' constant
  var readBuffer = UnsafeMutablePointer<UInt8>.allocate(capacity: 1)
  ~~~ ^
  let
[67/97] Compiling SwiftProtobuf ExtensionFields.swift
/tmp/tmplgknqkw3/swift-install/package/.build/checkouts/swift-protobuf/Sources/SwiftProtobuf/BinaryDelimited.swift:198:7: warning: variable 'readBuffer' was never mutated; consider changing to 'let' constant
  var readBuffer = UnsafeMutablePointer<UInt8>.allocate(capacity: 1)
  ~~~ ^
  let
[68/97] Compiling SwiftProtobuf ExtensionMap.swift
/tmp/tmplgknqkw3/swift-install/package/.build/checkouts/swift-protobuf/Sources/SwiftProtobuf/BinaryDelimited.swift:198:7: warning: variable 'readBuffer' was never mutate

        var grid_y  = Tensor<Float>(rangeFrom: 0, to: Float(h), stride: 1)
        ~~~ ^
        let
/notebooks/YOLOv4/Sources/YOLOv4.swift:42:13: warning: variable 'x_offset' was never mutated; consider changing to 'let' constant
        var x_offset = grid_y.reshaped(to: [1,1,w,1,1]).tiled(multiples: [1,h,1,anc,1])
        ~~~ ^
        let
/notebooks/YOLOv4/Sources/YOLOv4.swift:43:13: warning: variable 'y_offset' was never mutated; consider changing to 'let' constant
        var y_offset = grid_x.reshaped(to: [1,h,1,1,1]).tiled(multiples: [1,1,w,anc,1])
        ~~~ ^
        let
/notebooks/YOLOv4/Sources/YOLOv4.swift:44:13: warning: variable 'offset' was never mutated; consider changing to 'let' constant
        var offset = x_offset.concatenated(with: y_offset, alongAxis: -1)
        ~~~ ^
        let
[73/98] Merging module YOLOv4
[74/98] Wrapping AST for YOLOv4 for debugging
[75/98] Compiling SwiftProtobuf TimeUtils.swift
[76/98] Compiling SwiftProtobuf UnknownStorage.swift
[77/98

In [2]:
let imageURL = URL(fileURLWithPath: "dog_512.jpg")
let img = Image(jpeg: imageURL)
//var im = img.tensor.reshaped(to: [1,32,32,3]) / 255
var im = img.tensor.reshaped(to: [1,512,512,3]) / 255
var yolov4 = YOLOv4<Float>(cls:80).load(fileAtPath: "yolov4.weights")
let y = (im |> yolov4)//.transposed(permutation: [0,3,1,2])
 //yolov4.convBN2.conv.filter.transposed(permutation: [3,2,0,1])
print(y)

[[[  0.0063974783,    0.021929685,    0.037130427, ...,     0.84948146,
        0.9050464,      0.9385976],
  [  0.0066751447,   0.0056383526,   0.0045777657, ...,      0.9596767,
        0.9533385,      0.9398015],
  [   0.015628543,    0.039455645,    0.055431284, ...,    0.026045866,
      0.016820304,    0.012395469],
  ...,
  [ 0.00091934204,  0.00061285496,   0.0011124313, ...,    0.007310033,
      0.022635251,    0.021229774],
  [  0.0028067231,    0.003250748,    0.003165245, ...,   0.0011175871,
     0.0019192994,   0.0012732744],
  [  0.0011200905,  0.00091671944,   0.0009913146, ...,   0.0018109083,
      0.002732575,   0.0018697679]]]


In [3]:
time(re: 1000) {im |> yolov4}

984027.791 ms


In [17]:
var output = y.split(sizes: [4096, 1024, 256], alongAxis: -1)
var final_input = output[0].reshaped(to: [1, 255, 16*4, 16*4])

In [18]:
var flt = final_input.transposed(permutation: [0,2,3,1]).flattened()
for w in 0...4 {
for i in 0...2 {
    var idx = i * 85 + 255 * w
    print(w,i, flt[idx...idx+3])
}}


0 0 [0.0063974783, 0.0066751447,  0.015628543,  0.016297298]
0 1 [0.010297131, 0.012544636,  0.02545634, 0.032854374]
0 2 [0.012085359,  0.01180432,   0.0476887, 0.030941376]
1 0 [ 0.021929685, 0.0056383526,  0.039455645,  0.013508055]
1 1 [0.022766564,   0.0117164,   0.0418176, 0.024785336]
1 2 [0.023988912, 0.011012311, 0.055378787, 0.022901874]
2 0 [ 0.037130427, 0.0045777657,  0.055431284,  0.012093501]
2 1 [0.036301333, 0.011571684, 0.064270586, 0.025246155]
2 2 [0.035993148, 0.009118937, 0.098927885, 0.020242237]
3 0 [ 0.054286674, 0.0038980506,   0.04045847,  0.010848597]
3 1 [ 0.05304127, 0.011279538,  0.07327077, 0.027917847]
3 2 [0.053284332, 0.008149819,  0.15559536, 0.021519462]
4 0 [ 0.070437886, 0.0041791494,  0.036594566,  0.011666611]
4 1 [ 0.06931566, 0.011549581,  0.06374556, 0.028030355]
4 2 [  0.06929229, 0.0093157785,    0.1348131,  0.023183191]


In [ ]:
(0,0,0): 0.00639748, 0.00667515, 0.01562854, 0.01629730
(0,0,1): 0.01029713, 0.01254463, 0.02545635, 0.03285440
(0,0,2): 0.01208535, 0.01180432, 0.04768874, 0.03094139
(1,0,0): 0.02192968, 0.00563835, 0.03945564, 0.01350806
(1,0,1): 0.02276656, 0.01171640, 0.04181760, 0.02478534
(1,0,2): 0.02398891, 0.01101232, 0.05537879, 0.02290188